In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'https://github.com/trending' 
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('h1.h3.lh-condensed')
headings[0]

<h1 class="h3 lh-condensed">
<a data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"TRENDING_REPOSITORIES_PAGE","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":39840932,"originating_url":"https://github.com/trending","user_id":null}}' data-hydro-click-hmac="b5c21c00a03199cc476d548acb97ab258f58b11b86fc31b42a24574f04824f67" data-view-component="true" href="/google/googletest">
<svg aria-hidden="true" class="octicon octicon-repo mr-1 color-fg-muted" data-view-component="true" height="16" version="1.1" viewbox="0 0 16 16" width="16">
<path d="M2 2.5A2.5 2.5 0 014.5 0h8.75a.75.75 0 01.75.75v12.5a.75.75 0 01-.75.75h-2.5a.75.75 0 110-1.5h1.75v-2h-8a1 1 0 00-.714 1.7.75.75 0 01-1.072 1.05A2.495 2.495 0 012 11.5v-9zm10.5-1V9h-8c-.356 0-.694.074-1 .208V2.5a1 1 0 011-1h8zM5 12.25v3.25a.25.25 0 00.4.2l1.45-1.087a.25.25 0 01.3 0L8.6 15.7a.25.25 0 00.4-.2v-3.25a.25.25 0 00-.25-.25h-3.5a.25.25 0 00-.25.25z" fill-r

In [5]:
len(headings)

25

In [6]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('h1.h3.lh-condensed')]

['google/googletest',
 'projectdiscovery/nuclei-templates',
 'digitalocean/nginxconfig.io',
 'flutter/flutter',
 'PaddlePaddle/PaddleOCR',
 'flutter/pinball',
 'wolfogre/go-pprof-practice',
 'supabase/supabase',
 'felipefialho/frontend-challenges',
 'flutter/samples',
 'alibaba/fastjson2',
 'florinpop17/app-ideas',
 'charmbracelet/bubbletea',
 'huggingface/transformers',
 'databricks-academy/data-engineering-with-databricks',
 'hectorqin/reader',
 'Azure/azure-rest-api-specs',
 'terra-money/core',
 'saltstack/salt',
 'PKUFlyingPig/cs-self-learning',
 'actions/virtual-environments',
 'jojoldu/junior-recruit-scheduler',
 'dotnet/aspnetcore',
 'danielgindi/Charts',
 'microsoft/unilm']

In [11]:
url = 'https://github.com/search?o=desc&q=stars:%3E1&s=forks&type=Repositories' 
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":14204342,"result":{"id":14204342,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxNDIwNDM0Mg==","model_name":"Repository","url":"https://github.com/jtleek/datasharing"},"originating_url":"https://github.com/search?o=desc&amp;q=stars:%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="43ffd03f9b9f100d5d4cff6300156df517f385d2cd83787e829219fd8b4df50e" href="/jtleek/datasharing">jtleek/datasharing</a>
</div>

In [10]:
soup.text

'\n\n\n\n\n\nRate limit · GitHub\n\n\n\n\n\nWhoa there!\nYou have exceeded a secondary rate limit.\n        Please wait a few minutes before you try again;\n        in some cases this may take up to an hour.\n      \n\nContact Support —\n        GitHub Status —\n        @githubstatus\n\n\n\n\n\n\n\n\n\n\n'

In [12]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['jtleek/datasharing',
 'rdpeng/ProgrammingAssignment2',
 'octocat/Spoon-Knife',
 'tensorflow/tensorflow',
 'twbs/bootstrap',
 'Pierian-Data/Complete-Python-3-Bootcamp',
 'nightscout/cgm-remote-monitor',
 'jwasham/coding-interview-university',
 'rdpeng/ExData_Plotting1',
 'github/docs']

In [13]:
url = 'https://github.com/search?o=desc&p=2&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [14]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":2,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":189621607,"result":{"id":189621607,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxODk2MjE2MDc=","model_name":"Repository","url":"https://github.com/github/docs"},"originating_url":"https://github.com/search?o=desc&amp;p=2&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="e0c9c413683b12336f4ab1b96034d225dd4b7213d00e333d6630186cc5935124" href="/github/docs">github/docs</a>
</div>

In [15]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['github/docs',
 'opencv/opencv',
 'EbookFoundation/free-programming-books',
 'eugenp/tutorials',
 'CyC2018/CS-Notes',
 'jackfrued/Python-100-Days',
 'firstcontributions/first-contributions',
 'torvalds/linux',
 'Snailclimb/JavaGuide',
 'facebook/react']

In [16]:
url = 'https://github.com/search?o=desc&p=3&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [17]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":3,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":15783450,"result":{"id":15783450,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxNTc4MzQ1MA==","model_name":"Repository","url":"https://github.com/jlord/patchwork"},"originating_url":"https://github.com/search?o=desc&amp;p=3&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="e9e156fd89fd4f4322570e4ca03c27e94962a4e53376501c1e4090099efa3850" href="/jlord/patchwork">jlord/patchwork</a>
</div>

In [18]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['jlord/patchwork',
 'TheAlgorithms/Python',
 'ant-design/ant-design',
 'barryclark/jekyll-now',
 'bitcoin/bitcoin',
 'angular/angular.js',
 'kamranahmedse/developer-roadmap',
 'PanJiaChen/vue-element-admin',
 'django/django',
 'mui/material-ui']

In [19]:
url = 'https://github.com/search?o=desc&p=4&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [20]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":4,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":85077558,"result":{"id":85077558,"global_relay_id":"MDEwOlJlcG9zaXRvcnk4NTA3NzU1OA==","model_name":"Repository","url":"https://github.com/kamranahmedse/developer-roadmap"},"originating_url":"https://github.com/search?o=desc&amp;p=4&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="325592ad3d339c70a816b7cd30e0febd1f1cffa99766992420bf7e3a272c2f94" href="/kamranahmedse/developer-roadmap">kamranahmedse/developer-roadmap</a>
</div>

In [21]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['kamranahmedse/developer-roadmap',
 'PanJiaChen/vue-element-admin',
 'DefinitelyTyped/DefinitelyTyped',
 'django/django',
 'mui/material-ui',
 'RedHatTraining/DO180-apps',
 'qmk/qmk_firmware',
 'apache/spark',
 'apache/dubbo',
 'google/it-cert-automation-practice']

In [22]:
url = 'https://github.com/search?o=desc&p=5&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [23]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":5,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":63537249,"result":{"id":63537249,"global_relay_id":"MDEwOlJlcG9zaXRvcnk2MzUzNzI0OQ==","model_name":"Repository","url":"https://github.com/facebook/create-react-app"},"originating_url":"https://github.com/search?o=desc&amp;p=5&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="a062b7ac7393e0c6659a6a3b8e0b0188fd221d8aae2c97257b9afef49fa07848" href="/facebook/create-react-app">facebook/create-react-app</a>
</div>

In [24]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['facebook/create-react-app',
 'airbnb/javascript',
 'git/git',
 'nodejs/node',
 'sindresorhus/awesome',
 'iluwatar/java-design-patterns',
 'python/cpython',
 'd3/d3',
 'scikit-learn/scikit-learn',
 'atralice/Curso.Prep.Henry']

In [3]:
url = 'https://github.com/search?o=desc&p=7&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":7,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":12791642,"result":{"id":12791642,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxMjc5MTY0Mg==","model_name":"Repository","url":"https://github.com/BVLC/caffe"},"originating_url":"https://github.com/search?o=desc&amp;p=7&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="ca60a6ad3c9032632c5b2989897b455af1ac1cd40c2e955fc0a697b9a304322f" href="/BVLC/caffe">BVLC/caffe</a>
</div>

In [5]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['BVLC/caffe',
 'apache/echarts',
 'CSSEGISandData/COVID-19',
 'jenkins-docs/simple-java-maven-app',
 'vercel/next.js',
 'home-assistant/core',
 'moby/moby',
 'ColorlibHQ/AdminLTE',
 'scm-ninja/starter-web',
 'xingshaocheng/architect-awesome']

In [6]:
url = 'https://github.com/search?o=desc&p=8&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":8,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":7512484,"result":{"id":7512484,"global_relay_id":"MDEwOlJlcG9zaXRvcnk3NTEyNDg0","model_name":"Repository","url":"https://github.com/ArduPilot/ardupilot"},"originating_url":"https://github.com/search?o=desc&amp;p=8&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="f601b80df1bdce61deeda313efce5c121a9eb1e642ac2468cf8d2c4ea0d00738" href="/ArduPilot/ardupilot">ArduPilot/ardupilot</a>
</div>

In [8]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['ArduPilot/ardupilot',
 'codebasics/py',
 'ageitgey/face_recognition',
 'bailicangdu/vue2-elm',
 'angular/angular-cli',
 'kdn251/interviews',
 'Trinea/android-open-project',
 'zero-to-mastery/start-here-guidelines',
 'FortAwesome/Font-Awesome',
 'Homebrew/legacy-homebrew']

In [9]:
url = 'https://github.com/search?o=desc&p=9&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":9,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":65388917,"result":{"id":65388917,"global_relay_id":"MDEwOlJlcG9zaXRvcnk2NTM4ODkxNw==","model_name":"Repository","url":"https://github.com/jakevdp/PythonDataScienceHandbook"},"originating_url":"https://github.com/search?o=desc&amp;p=9&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="739f5bacd6b2f2da309c9ae8bf4927a9384c5739cdc0235464bda91e366d620f" href="/jakevdp/PythonDataScienceHandbook">jakevdp/PythonDataScienceHandbook</a>
</div>

In [11]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['jakevdp/PythonDataScienceHandbook',
 'aymericdamien/TensorFlow-Examples',
 'pallets/flask',
 'github/opensource.guide',
 'golang/go',
 'TheOdinProject/css-exercises',
 'selfteaching/the-craft-of-selfteaching',
 'pandas-dev/pandas',
 'ElemeFE/element',
 'ionic-team/ionic-framework']

In [15]:
url = 'https://github.com/search?o=desc&p=10&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [16]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":10,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":151834062,"result":{"id":151834062,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxNTE4MzQwNjI=","model_name":"Repository","url":"https://github.com/doocs/advanced-java"},"originating_url":"https://github.com/search?o=desc&amp;p=10&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="a1b5dbd4a88b4054862a755cc84a8806667ebc0e3b2996a0aed4f405b05dd362" href="/doocs/advanced-java">doocs/advanced-java</a>
</div>

In [17]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['doocs/advanced-java',
 'MarlinFirmware/Marlin',
 'shadowsocks/shadowsocks-windows',
 'CoreyMSchafer/code_snippets',
 'MicrosoftDocs/azure-docs',
 'odoo/odoo',
 'FreeRDP/FreeRDP',
 'hakimel/reveal.js',
 'gabrielecirulli/2048',
 'udacity/course-collaboration-travel-plans']

In [18]:
url = 'https://github.com/search?o=desc&p=11&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [19]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":11,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":19415064,"result":{"id":19415064,"global_relay_id":"MDEwOlJlcG9zaXRvcnkxOTQxNTA2NA==","model_name":"Repository","url":"https://github.com/ossu/computer-science"},"originating_url":"https://github.com/search?o=desc&amp;p=11&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="be409c79a21bec307c6a75bc34ac714763fdb263fb6a4b138e88c09e0b07bc45" href="/ossu/computer-science">ossu/computer-science</a>
</div>

In [20]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['ossu/computer-science',
 'jakevdp/PythonDataScienceHandbook',
 'aymericdamien/TensorFlow-Examples',
 'reduxjs/redux',
 'pallets/flask',
 'TheAlgorithms/Java',
 'scutan90/DeepLearning-500-questions',
 'huggingface/transformers',
 'github/opensource.guide',
 'golang/go']

In [24]:
url = 'https://github.com/search?o=desc&p=12&q=stars%3A%3E1&s=forks&type=Repositories'
response = get(url)

In [25]:
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('div.f4.text-normal')
headings[0]

<div class="f4 text-normal">
<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":12,"per_page":10,"query":"stars:&gt;1","result_position":1,"click_id":36535156,"result":{"id":36535156,"global_relay_id":"MDEwOlJlcG9zaXRvcnkzNjUzNTE1Ng==","model_name":"Repository","url":"https://github.com/reduxjs/redux"},"originating_url":"https://github.com/search?o=desc&amp;p=12&amp;q=stars%3A%3E1&amp;s=forks&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="eff2a06d7f616754e6648970e679e745ee63b926291636a62d8b6613bcc6bac6" href="/reduxjs/redux">reduxjs/redux</a>
</div>

In [26]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('div.f4.text-normal')]

['reduxjs/redux',
 'pallets/flask',
 'scutan90/DeepLearning-500-questions',
 'huggingface/transformers',
 'github/opensource.guide',
 'golang/go',
 'TheOdinProject/css-exercises',
 'selfteaching/the-craft-of-selfteaching',
 'netty/netty',
 'Azure/azure-quickstart-templates']

In [23]:
soup.text

'\n\n\n\n\n\nRate limit · GitHub\n\n\n\n\n\nWhoa there!\nYou have exceeded a secondary rate limit.\n        Please wait a few minutes before you try again;\n        in some cases this may take up to an hour.\n      \n\nContact Support —\n        GitHub Status —\n        @githubstatus\n\n\n\n\n\n\n\n\n\n\n'

In [ ]:
import pandas as pd
import os

#### Pulled from class will need to add values

"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = [
 
 'topkecleon/telegram-bot-bash',
 'evil-mad/EggBot',
 'telegram-bot-rb/telegram-bot',
 'alfficcadenti/splinterlands-bot',
 'nodeWechat/wechat4u',
 'tucnak/telebot',
 'CarlGroth/Carl-Bot',
 'grapeot/WechatForwardBot',
 'googleworkspace/hangouts-chat-samples',
 'GreyWolfDev/Werewolf',
 'abdelhai/awesome-bots',
 'mdgspace/bot',
 'Sank6/Discord-Bot-List',
 'samc621/SneakerBot',
 'boto/boto3-sample',
 'huseinzol05/Stock-Prediction-Models',
 'SAPConversationalAI/Webchat',
 'soumyadityac/youtube-viewer',
 'CodeWithJoe2020/pancakeswapBot',
 'Merubokkusu/discord-spam-bots',
 'ZeroDiscord/EconomyBot',
 'scrapinghub/slackbot',
 'baidu/boteye',
 'kcloze/swoole-bot',
 'hyperchessbot/hyperbot',
 'yangyuan/hearthrock',
]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data():
    """
    This function checks for a local csv file of raw data. If a local csv exists it reads it 
    to a df.
    
    If no local csv of raw data exists, it loops through all of the repos in the REPOS list above and
    and runs the process functions to scrape the repo name, language, and readme
    contents. It then writes the resulting dictionary of lists to a df, returning
    a dataframe of raw data. It then caches the raw data to a local csv file.
    """
    if os.path.isfile('raw_data.csv'):
        df = pd.read_csv('raw_data.csv')

    else:
        df = pd.DataFrame([process_repo(repo) for repo in REPOS])
        df.to_csv('raw_data.csv')

    return df


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)

In [28]:
df = pd.read_json('data.json')
df.head()

,repo,language,readme_contents
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...
3,jtleek/datasharing,None,How to share data with a statistician\n=======...
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...


In [30]:
df.language.value_counts(dropna=False)

JavaScript          62
Python              49
NaN                 32
Java                21
TypeScript          19
C++                 17
Go                  14
HTML                12
Jupyter Notebook    11
C                    9
CSS                  6
C#                   6
Dart                 6
Vue                  5
PHP                  5
Shell                5
PowerShell           2
Ruby                 2
Rich Text Format     2
R                    2
Kotlin               2
Swift                2
Jinja                1
Lua                  1
Objective-C          1
Scala                1
CoffeeScript         1
Clojure              1
OCaml                1
Solidity             1
Groovy               1
ApacheConf           1
Name: language, dtype: int64

In [31]:
df.shape

(301, 3)

In [2]:
# unicode, regex, json for text digestion
import unicodedata
import re
import json

# nltk: natural language toolkit -> tokenization, stopwords (more on this soon)
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# pandas dataframe manipulation, acquire script, time formatting
import pandas as pd
import acquire
from time import strftime

# shh, down in front
import warnings
warnings.filterwarnings('ignore')

In [3]:
def basic_clean(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r"[^a-z0-9'\s]", '', text)
    return text

def tokenize(text):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    text = tokenizer.tokenize(text, return_str=True)
    return text

def stem(text):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in text.split()]
    text = ' '.join(stems)
    return text

def lemmatize(text):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    text = ' '.join(lemmas)
    return text



In [35]:
df['clean'] = df.readme_contents.apply(basic_clean)
tokenized_df = df.clean.apply(tokenize)
df['stemmed'] = tokenized_df.apply(stem)
df.head()

,repo,language,readme_contents,clean,stemmed
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...,codeup setup script\n\nsetup script for codeu...,codeup setup script setup script for codeup st...
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movies application\n\nfor this project we wil...,movi applic for thi project we will be build a...
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel\n\n\nthere are several guides for...,linux kernel there are sever guid for kernel d...
3,jtleek/datasharing,None,How to share data with a statistician\n=======...,how to share data with a statistician\n\n\nthi...,how to share data with a statistician thi is a...
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction\n\nthis second programming assig...,introduct thi second program assign will requi...


In [36]:
type(df.clean)

pandas.core.series.Series

In [37]:
type(df.readme_contents)

pandas.core.series.Series

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             301 non-null    object
 1   language         269 non-null    object
 2   readme_contents  301 non-null    object
 3   clean            301 non-null    object
 4   stemmed          301 non-null    object
dtypes: object(5)
memory usage: 11.9+ KB


In [4]:
import matt_prepare

In [5]:
df = matt_prepare.clean_df()

In [6]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized,character_count,word_count
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,1062,119
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movies application project building single pag...,movi applic project build singl page movi appl...,movie application project building single page...,2888,411
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel several guides kernel developers ...,linux kernel sever guid kernel develop user gu...,linux kernel several guide kernel developer us...,505,58
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,introduct second program assign requir write r...,introduction second programming assignment req...,2488,332
5,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,well hello repositori meant provid exampl fork...,well hello repository meant provide example fo...,519,60


In [7]:
df.language.value_counts(dropna=False)

JavaScript          62
Python              49
Java                21
TypeScript          19
C++                 17
Go                  14
HTML                12
Jupyter Notebook    11
C                    9
CSS                  6
C#                   6
Dart                 6
Vue                  5
PHP                  5
Shell                5
PowerShell           2
Ruby                 2
Rich Text Format     2
R                    2
Kotlin               2
Swift                2
Jinja                1
Lua                  1
Objective-C          1
Scala                1
CoffeeScript         1
Clojure              1
OCaml                1
Solidity             1
Groovy               1
ApacheConf           1
Name: language, dtype: int64

In [8]:
df.shape

(269, 8)

In [9]:
df = df.dropna()

In [10]:
df.shape

(269, 8)

In [11]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized,character_count,word_count
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,1062,119
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movies application project building single pag...,movi applic project build singl page movi appl...,movie application project building single page...,2888,411
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel several guides kernel developers ...,linux kernel sever guid kernel develop user gu...,linux kernel several guide kernel developer us...,505,58
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,introduct second program assign requir write r...,introduction second programming assignment req...,2488,332
5,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,well hello repositori meant provid exampl fork...,well hello repository meant provide example fo...,519,60


In [16]:
import numpy as np
top_five = ['JavaScript', 'Python', 'Java', 'TypeScript', 'C++']
df['top_five_languages'] = np.where(df.language.isin(top_five), df.language, 'other')
df.top_five.value_counts()

other         101
JavaScript     62
Python         49
Java           21
TypeScript     19
C++            17
Name: top_five, dtype: int64

In [158]:
import matt_prepare
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# unicode, regex, json for text digestion
import unicodedata
import re
import json

# nltk: natural language toolkit -> tokenization, stopwords (more on this soon)
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

# pandas dataframe manipulation, acquire script, time formatting
import pandas as pd
import acquire
from time import strftime

# shh, down in front
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
df = matt_prepare.clean_df()
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized,character_count,word_count,top_five_languages
0,gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,codeup setup script setup script codeup studen...,1062,119,other
1,gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w...",movies application project building single pag...,movi applic project build singl page movi appl...,movie application project building single page...,2888,411,JavaScript
2,torvalds/linux,C,Linux kernel\n============\n\nThere are severa...,linux kernel several guides kernel developers ...,linux kernel sever guid kernel develop user gu...,linux kernel several guide kernel developer us...,505,58,other
4,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,introduct second program assign requir write r...,introduction second programming assignment req...,2488,332,other
5,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,well hello repositori meant provid exampl fork...,well hello repository meant provide example fo...,519,60,other


In [3]:
df.top_five_languages.value_counts()

other         101
JavaScript     62
Python         49
Java           21
TypeScript     19
C++            17
Name: top_five_languages, dtype: int64

In [95]:
#baseline
101 / df.shape[0]

0.3754646840148699

In [101]:
df.shape

(269, 9)

In [4]:
train, validate, test = matt_prepare.split_data(df)
train.shape, validate.shape, test.shape

((150, 9), (65, 9), (54, 9))

In [6]:
X_train = train.lemmatized
y_train = train.top_five_languages
X_train.head()

252    py repository store sample python program repo...
119    hrefhttpsreduxjsorgimg srchttpscamogithubuserc...
240    deprecated use floatingactionbuttonhttpsdevelo...
69     h1 stylecolor redthis repo deprecated due chan...
95     threejs npm packagenpmnpmurl build sizebuildsi...
Name: lemmatized, dtype: object

In [8]:
y_train.head()

252         other
119    TypeScript
240          Java
69         Python
95     JavaScript
Name: top_five_languages, dtype: object

In [9]:
X_train = train.lemmatized
y_train = train.top_five_languages
X_val = validate.lemmatized
y_val = validate.top_five_languages
X_test = test.lemmatized
y_test = test.top_five_languages

In [12]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow

<150x21147 sparse matrix of type '<class 'numpy.int64'>'
	with 45581 stored elements in Compressed Sparse Row format>

In [13]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf

<150x21147 sparse matrix of type '<class 'numpy.float64'>'
	with 45581 stored elements in Compressed Sparse Row format>

In [14]:
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.8066666666666666

In [15]:
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train)

0.8133333333333334

In [16]:
tree = DecisionTreeClassifier(max_depth=10)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.9066666666666666

In [17]:
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train)

0.9266666666666666

In [18]:
tree = DecisionTreeClassifier(max_depth=15)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.94

In [19]:
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train)

0.98

In [20]:
tree = DecisionTreeClassifier(max_depth=20)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.9933333333333333

In [21]:
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train)

0.9866666666666667

In [109]:
#cv train
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)
#tfidf train
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)
#DT
tree = DecisionTreeClassifier(max_depth=15, random_state=123)
#tree fit
tree.fit(X_bow, y_train)

DecisionTreeClassifier(max_depth=15, random_state=123)

In [110]:
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9666666666666667, 0.6307692307692307)

In [111]:
tree.fit(X_tfidf, y_train)

DecisionTreeClassifier(max_depth=15, random_state=123)

In [112]:
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.98, 0.6)

In [113]:
#cv train
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)
#tfidf train
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)
#DT
tree = DecisionTreeClassifier(max_depth=20, random_state=123)
#tree fit
tree.fit(X_bow, y_train)

DecisionTreeClassifier(max_depth=20, random_state=123)

In [114]:
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9933333333333333, 0.6153846153846154)

In [115]:
tree.fit(X_tfidf, y_train)

DecisionTreeClassifier(max_depth=20, random_state=123)

In [116]:
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.9933333333333333, 0.5692307692307692)

In [117]:
#cv train
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)
#tfidf train
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)
#DT
tree = DecisionTreeClassifier(max_depth=10, random_state=123)
#tree fit
tree.fit(X_bow, y_train)

DecisionTreeClassifier(max_depth=10, random_state=123)

In [118]:
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9133333333333333, 0.6615384615384615)

In [119]:
tree.fit(X_tfidf, y_train)

DecisionTreeClassifier(max_depth=10, random_state=123)

In [120]:
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.9266666666666666, 0.6)

In [121]:
#cv train
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)
#tfidf train
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)
#DT
tree = DecisionTreeClassifier(max_depth=30, random_state=123)
#tree fit
tree.fit(X_bow, y_train)

DecisionTreeClassifier(max_depth=30, random_state=123)

In [122]:
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(1.0, 0.6)

In [123]:
tree.fit(X_tfidf, y_train)

DecisionTreeClassifier(max_depth=30, random_state=123)

In [124]:
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(1.0, 0.5692307692307692)

## Best DT single words 

In [128]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)

In [129]:
tree = DecisionTreeClassifier(max_depth=15, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9666666666666667, 0.6307692307692307)

In [130]:
tree = DecisionTreeClassifier(max_depth=15, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.98, 0.6)

---

In [131]:
cv = CountVectorizer(ngram_range=(1,2))
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

tfidf = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)

In [132]:
tree = DecisionTreeClassifier(max_depth=10, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9466666666666667, 0.5846153846153846)

In [133]:
tree = DecisionTreeClassifier(max_depth=10, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.88, 0.6307692307692307)

In [134]:
tree = DecisionTreeClassifier(max_depth=20, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9933333333333333, 0.5846153846153846)

In [135]:
tree = DecisionTreeClassifier(max_depth=20, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.9933333333333333, 0.5846153846153846)

In [136]:
tree = DecisionTreeClassifier(max_depth=30, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(1.0, 0.5692307692307692)

In [137]:
tree = DecisionTreeClassifier(max_depth=30, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(1.0, 0.5846153846153846)

In [138]:
tree = DecisionTreeClassifier(max_depth=40, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(1.0, 0.5692307692307692)

In [139]:
tree = DecisionTreeClassifier(max_depth=40, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(1.0, 0.5846153846153846)

In [140]:
tree = DecisionTreeClassifier(max_depth=50, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(1.0, 0.5692307692307692)

In [141]:
tree = DecisionTreeClassifier(max_depth=15, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.9666666666666667, 0.5692307692307692)

### Best DT ngram range 1,2

In [142]:
cv = CountVectorizer(ngram_range=(1,2))
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

tfidf = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)

In [143]:
tree = DecisionTreeClassifier(max_depth=20, random_state=123)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train), tree.score(X_bow_val, y_val)

(0.9933333333333333, 0.5846153846153846)

In [144]:
tree = DecisionTreeClassifier(max_depth=10, random_state=123)
tree.fit(X_tfidf, y_train)
tree.score(X_tfidf, y_train), tree.score(X_tfidf_val, y_val)

(0.88, 0.6307692307692307)

In [146]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)

In [149]:
lm = LogisticRegression(random_state=123).fit(X_bow, y_train)
lm.score(X_bow, y_train), lm.score(X_bow_val, y_val)

(0.9933333333333333, 0.5384615384615384)

In [150]:
lm = LogisticRegression(random_state=123).fit(X_tfidf, y_train)
lm.score(X_tfidf, y_train), lm.score(X_tfidf_val, y_val)

(0.8, 0.5076923076923077)

In [151]:
cv = CountVectorizer(ngram_range=(1,2))
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)

In [152]:
lm = LogisticRegression(random_state=123).fit(X_bow, y_train)
lm.score(X_bow, y_train), lm.score(X_bow_val, y_val)

(0.9933333333333333, 0.5846153846153846)

In [153]:
lm = LogisticRegression(random_state=123).fit(X_tfidf, y_train)
lm.score(X_tfidf, y_train), lm.score(X_tfidf_val, y_val)

(0.8, 0.5076923076923077)

In [154]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)

# Evaluate Logistic Regression models on train & validate set by looping through different values for c hyperparameter

# create empty list for which to append metrics from each loop
metrics = []

# create loop for values in list
for c in [.001, .005, .01, .05, .1, .5, 1, 5, 10, 50, 100, 500, 1000]:
            
    # define the model setting hyperparameters to values for current loop
    logit = LogisticRegression(C=c)
    
    # fit the model on train
    logit.fit(X_bow, y_train)
    
    # use the model and evaluate performance on train
    train_accuracy = logit.score(X_bow, y_train)
    # use the model and evaluate performance on validate
    validate_accuracy = logit.score(X_bow_val, y_val)
    
    # create output of current loop's hyperparameters and accuracy to append to metrics
    output = {
        'C': c,
        'train_accuracy': train_accuracy,
        'validate_accuracy': validate_accuracy
    }
    
    metrics.append(output)

# convert metrics list to a dataframe for easy reading
df = pd.DataFrame(metrics)
# add column to assess the difference between train & validate accuracy
df['difference'] = df.train_accuracy - df.validate_accuracy
df

,C,train_accuracy,validate_accuracy,difference
0,0.001,0.733333,0.492308,0.241026
1,0.005,0.886667,0.569231,0.317436
2,0.010,0.926667,0.615385,0.311282
3,0.050,0.953333,0.646154,0.307179
4,0.100,0.966667,0.569231,0.397436
5,0.500,0.993333,0.553846,0.439487
6,1.000,0.993333,0.538462,0.454872
7,5.000,1.000000,0.553846,0.446154
8,10.000,1.000000,0.553846,0.446154
9,50.000,1.000000,0.538462,0.461538


In [156]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X_train)
X_tfidf_val = tfidf.transform(X_val)
# Evaluate Logistic Regression models on train & validate set by looping through different values for c hyperparameter

# create empty list for which to append metrics from each loop
metrics = []

# create loop for values in list
for c in [.001, .005, .01, .05, .1, .5, 1, 5, 10, 50, 100, 500, 1000]:
            
    # define the model setting hyperparameters to values for current loop
    logit = LogisticRegression(C=c)
    
    # fit the model on train
    logit.fit(X_tfidf, y_train)
    
    # use the model and evaluate performance on train
    train_accuracy = logit.score(X_tfidf, y_train)
    # use the model and evaluate performance on validate
    validate_accuracy = logit.score(X_tfidf_val, y_val)
    
    # create output of current loop's hyperparameters and accuracy to append to metrics
    output = {
        'C': c,
        'train_accuracy': train_accuracy,
        'validate_accuracy': validate_accuracy
    }
    
    metrics.append(output)

# convert metrics list to a dataframe for easy reading
df = pd.DataFrame(metrics)
# add column to assess the difference between train & validate accuracy
df['difference'] = df.train_accuracy - df.validate_accuracy
df

,C,train_accuracy,validate_accuracy,difference
0,0.001,0.360000,0.400000,-0.040000
1,0.005,0.360000,0.400000,-0.040000
2,0.010,0.360000,0.400000,-0.040000
3,0.050,0.360000,0.400000,-0.040000
4,0.100,0.360000,0.400000,-0.040000
5,0.500,0.693333,0.476923,0.216410
6,1.000,0.800000,0.507692,0.292308
7,5.000,0.993333,0.676923,0.316410
8,10.000,0.993333,0.692308,0.301026
9,50.000,0.993333,0.661538,0.331795


In [159]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
X_bow_val = cv.transform(X_val)


# Evaluate KNearest Neighbors models on train & validate set by looping through different values for k hyperparameter

# create empty list for which to append scores from each loop
scores = []
k_range = range (1,20)
# create loop for range 1-20
for k in k_range:
            
    # define the model setting hyperparameters to values for current loop
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # fit the model on train
    knn.fit(X_bow, y_train)
    
    # use the model and evaluate performance on train
    train_accuracy = knn.score(X_bow, y_train)
    # use the model and evaluate performance on validate
    validate_accuracy = knn.score(X_bow_val, y_val)
    
    # create output of current loop's hyperparameters and accuracy to append to metrics
    output = {
        "k": k,
        "train_accuracy": train_accuracy,
        "validate_accuracy": validate_accuracy
    }
    
    scores.append(output)

# convert scores list to a dataframe for easy reading
df = pd.DataFrame(scores)
# add column to assess the difference between train & validate accuracy
df['difference'] = df.train_accuracy - df.validate_accuracy
df

,k,train_accuracy,validate_accuracy,difference
0,1,1.000000,0.323077,0.676923
1,2,0.520000,0.276923,0.243077
2,3,0.593333,0.292308,0.301026
3,4,0.660000,0.384615,0.275385
4,5,0.573333,0.415385,0.157949
5,6,0.540000,0.415385,0.124615
6,7,0.446667,0.338462,0.108205
7,8,0.546667,0.400000,0.146667
8,9,0.586667,0.523077,0.063590
9,10,0.600000,0.430769,0.169231
